In [2]:
from whoosh.qparser import QueryParser, MultifieldParser
from whoosh import index, sorting
from whoosh.highlight import UppercaseFormatter
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
import html
import re
import nltk

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\alfre\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alfre\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [56]:
# Open an existing index
ix = index.open_dir("indexdir", indexname="dblp")

#Search fields
fields = ["author", "publication_venue", "title", "year"]

# Create a query parser that looks in the "title", "author", "year" "publication venue" fields
#parser = QueryParser(fields, ix.schema)
parser = MultifieldParser(fields, schema = ix.schema)

stop_words = set(stopwords.words('english'))


# Parse a query
search_string = input('What would you like to search?')
if search_string == "":
    print("Please enter a valid phrase.")
else:
    for word in search_string:
        word = word_tokenize(search_string)
        word = [w.lower() for w in word if not w.lower() in stop_words]
    search_string_normalized = " ".join(word)
    #print(search_string_normalized)

    query = parser.parse(search_string_normalized)
    #print(query)

    # Use the searcher to find documents that match the query
    N = 100
    #highlighter = UppercaseFormatter()
    with ix.searcher() as searcher:
        # Create a results object to hold the top N results
        results = searcher.search(query, limit=N)
        # Print the rank, score, docID, and snippet for each result
        if results.scored_length() > 0:
            for hit in results:
                #print(hit)
                print(f"Rank: {hit.rank+1}, Score: {hit.score}, DocID: {hit.docnum}")
                for field in fields:
                    if hit.highlights(field) != "":
                        highlighted_text = re.sub('<.*?>', '', hit.highlights(field))
                        print("Matched field: ", "<", field, ">")
                        print("Here is an excerpt: ", '"', highlighted_text,'"')
                print('\n')
        else:
            print("There is no match for your query. Please try something else.")

Rank: 1, Score: 13.704622370486668, DocID: 94939
Matched field:  < title >
Here is an excerpt:  " Jahresrückblick 2022 "


Rank: 2, Score: 13.704622370486668, DocID: 106362
Matched field:  < title >
Here is an excerpt:  " Vorwort 2022 "


Rank: 3, Score: 13.230932720450442, DocID: 79855
Matched field:  < title >
Here is an excerpt:  " Videogames, CG 2022 and the 2022 WCCC "


Rank: 4, Score: 12.883637303511994, DocID: 80369
Matched field:  < title >
Here is an excerpt:  " COMPSAC 2022: A Retrospective "


Rank: 5, Score: 12.883637303511994, DocID: 113848
Matched field:  < title >
Here is an excerpt:  " The 2022 Computer Olympiad "


Rank: 6, Score: 12.573905893755507, DocID: 316909
Matched field:  < title >
Here is an excerpt:  " EASE 2022: The International...Software Engineering 2022, Gothenburg, Sweden...13 - 15, 2022, EASE "


Rank: 7, Score: 11.794993473817504, DocID: 324492
Matched field:  < title >
Here is an excerpt:  " Theory, CWIT 2022, Ottawa, ON, Canada, June...5-8, 2022, C

In [50]:
from whoosh.qparser import QueryParser
from whoosh import scoring

ix = index.open_dir("indexdir", indexname="dblp")
# Assuming 'index' is your index
with ix.searcher() as searcher:
    query = QueryParser("year", ix.schema).parse("2023")
    print(query)
    results = searcher.search(query)
    for hit in results:
        print(hit)


year:b'\x00\x80\x00\x07\xe7'
<Hit {'author': 'Tuncer I. Ören, Paul K. Davis, Rhys Goldstein, Azam Khan, Laurent Capocchi, Maâmar El-Amine Hamri, Navonil Mustafee, Alison Harper, Baocun Hou, Bo Hu Li 0001, Yang Liu', 'publication_venue': '', 'title': 'Simulation as Experimentation., Body of Knowledge for Modeling and Simulation', 'year': '2023'}>
<Hit {'author': 'Tuncer I. Ören, Umut Durak, Ernest H. Page, Andreas Tolk, Saikou Y. Diallo', 'publication_venue': '', 'title': 'Simulation as Experience to Enhance Three Types of Skills., Body of Knowledge for Modeling and Simulation', 'year': '2023'}>
<Hit {'author': 'Tuncer I. Ören, Valdemar Vicente Graciano Neto, Paul K. Davis, Bernard P. Zeigler', 'publication_venue': '', 'title': 'Reliability and Quality Assurance of M&S., Body of Knowledge for Modeling and Simulation', 'year': '2023'}>
<Hit {'author': 'Emilian Mihailov, Cristina Voinea, Constantin Vica', 'publication_venue': 'Sci. Eng. Ethics', 'title': 'Is Online Moral Outrage Outrageou

In [17]:
# Assuming 'index' is your index
ix = index.open_dir('indexdir', indexname = 'dblp')
ix = ix.optimize()
num_docs = ix.doc_count_all()
print(f"The index contains {num_docs} documents.")
'''
with ix.searcher() as searcher:
    # Get all documents in the index
    all_docs = searcher.documents()
    
    # Print all documents
    for doc in all_docs:
        print(doc)
'''


AttributeError: 'NoneType' object has no attribute 'doc_count_all'

In [ ]:

# Create a query parser that looks in the "title", "author", "year" "publication venue" fields
parser = MultifieldParser(["author", "publication_venue", "title", "year"], ix.schema)

# Parse a query
query = parser.parse("2022")
print(query)

# Use the searcher to find documents that match the query
N = 5
searcher = ix.searcher
with ix.searcher() as searcher:
    # Create a results object to hold the top N results
    results = searcher.search(query, limit=N)
    print(results)

    # Print the rank, score, docID, and snippet for each result
    for hit in results:
        print(f"Rank: {hit.rank}, Score: {hit.score}, DocID: {hit.docnum}")
        print(f"Snippet: {hit.highlights('content')}")
